In [5]:
import os
import numpy as np
import pandas as pd
from zipfile import ZipFile
from kaggle.api.kaggle_api_extended import KaggleApi
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image

In [7]:
dataset_path = "./utkface-new"

os.makedirs(dataset_path, exist_ok=True)

api = KaggleApi()
api.authenticate()
print("Downloading dataset...")
api.dataset_download_files('jangedoo/utkface-new', path=dataset_path, unzip=True)
print("Dataset downloaded and extracted to:", dataset_path)



Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
Dataset downloaded and extracted to: ./utkface-new


In [ ]:
dataset_path = "path_to_utkface_dataset"

images = []
ages = []

for file_name in os.listdir(dataset_path):
    if file_name.endswith(".jpg"):
        age = int(file_name.split("_")[0])
        img_path = os.path.join(dataset_path, file_name)
        img = Image.open(img_path).resize((224,224))
        img = np.array(img)
        if img.shape == (224,224,3):  # مطمئن شدن از RGB بودن تصویر
            images.append(img)
            ages.append(age)

images = np.array(images, dtype="float32")
ages = np.array(ages, dtype="float32")